In [2]:
import pandas as pd
import os
import re
import codecs
import requests
from bs4 import BeautifulSoup
    
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

In [9]:
def text_file_to_dataframe(filename):
    # Check if the text file exists
    if os.path.exists(filename):
        
        # Read the text file and create a list of lines
        with open(filename, "r") as file:
            lines = file.readlines()
            content = [line.strip() for line in lines]

        # Create a DataFrame from the list
        df = pd.DataFrame(content, columns=['links'])
        
        return df
    else:
        print(f"{filename} does not exist.")
        return None

In [16]:
def get_n_review_by_xpath(driver, xpath_tag_dictionary, index, df, current_review_index):
    # S E L E N I U M
    review_body_tag = xpath_tag_dictionary['review'][current_review_index]
    review_body_element = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, review_body_tag)))
    review_body_text = review_body_element.text
    print(review_body_text)
    # df[f'review {index}'].iloc[i] = review_body_text
    return df

In [15]:
def get_5_book_reviews_metadata(df, link, driver, index, number_of_reviews=5):
    for current_review_index in range(number_of_reviews):
        xpath_tag_dictionary = {'review': ['//*[@id="ReviewsSection"]/div[5]/div[2]/div[1]/article/section/section[2]/section/div/div[1]/span',                                          
                                           '//*[@id="ReviewsSection"]/div[5]/div[2]/div[3]/article/section/section[2]/section/div/div[1]/span',                                          
                                           '//*[@id="ReviewsSection"]/div[5]/div[2]/div[4]/article/section/section[2]/section/div/div[1]/span',                                          
                                           '//*[@id="ReviewsSection"]/div[5]/div[2]/div[5]/article/section/section[2]/section/div/div[1]/span',                                          
                                           '//*[@id="ReviewsSection"]/div[5]/div[2]/div[6]/article/section/section[2]/section/div/div[1]/span'                                          
                                          ]}
        # Flag track success
        review_found = False
        try:
            # M A I N
            df = get_n_review_by_xpath(driver, xpath_tag_dictionary, index, df, current_review_index)
            review_found = True
        except (TimeoutException, NoSuchElementException):
            print(f"Could not get review {current_review_index + 1} for book {index + 1} searching for \"more button\": \n{link}")
            print("\n")
            review_show_more_button_tag = '/html/body/div[1]/div/main/div[1]/div[2]/div[3]/div/div[5]/div[2]/div[1]/article/section/section[2]/section/div/div[2]/div/button/'
            try:
                # B U T T O N   C L I C K
                review_show_more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, review_show_more_button_tag)))
                review_show_more_button.click()
            except (TimeoutException, NoSuchElementException):
                print(f"Could not find show more button for Book {index + 1} Review {current_review_index + 1}: \n{link}")
                print("\n")
            except ElementClickInterceptedException:
                print(f"Could not click on show more button for Book {index + 1} Review {current_review_index + 1}: \n{link}")
                print("\n")
        else:
            try:
                # M A I N 2 N D   T R Y
                df = get_n_review_by_xpath(driver, xpath_tag_dictionary, index, df, current_review_index)
                review_found = True
            except (TimeoutException, NoSuchElementException):
                print(f"Could not get review {current_review_index + 1} for book {index + 1} after clicking \"more\" button \n{link}")
                print("\n")

        if review_found:
            continue
        else:
            break
    return df

In [6]:
def scrape_random_books_selenium(df, num_loops):
    driver = webdriver.Chrome()
    
    for i in range(num_loops):
        link = df['links'].iloc[i]
        driver.get(link)
        
        # MAIN
        df = get_5_book_reviews_metadata(df, link, driver, i)

        # O T H E R   M E T A D A T A
        # df = get_metadata()
        
    driver.quit()
    return df

In [10]:
input_filename = "links.txt"
df = text_file_to_dataframe(input_filename)

In [13]:
df

,links
0,https://www.goodreads.com/book/show/13771831-b...
1,https://www.goodreads.com/book/show/28111953-j...
2,https://www.goodreads.com/book/show/24612758-m...
3,https://www.goodreads.com/book/show/25946661-l...
4,https://www.goodreads.com/book/show/57566.Secr...
...,...
469,https://www.goodreads.com/book/show/61330903-c...
470,https://www.goodreads.com/book/show/8073047-go...
471,https://www.goodreads.com/book/show/12820471-m...
472,https://www.goodreads.com/book/show/17205.Crys...


In [14]:
df = scrape_random_books_selenium(df,5)

Not the greatest until about the last third of the book. Maybe my taste in authors is changing.
Not the greatest until about the last third of the book. Maybe my taste in authors is changing.
This book reminded me of The Golden Girls television series. It included some of the more recent references to people and events that have occured since the series.
This book reminded me of The Golden Girls television series. It included some of the more recent references to people and events that have occured since the series.
fern michaels never fails to provide a story that you just cannot put down, I cant wait for her next one
fern michaels never fails to provide a story that you just cannot put down, I cant wait for her next one
Another good read!
Another good read!
Another good read!
Another good read!
Could not get review 1 for book 2 searching for "more button": 
https://www.goodreads.com/book/show/28111953-jokes-jokes-and-more-jokes?from_search=true&from_srp=true&qid=ZpnjM8vxLv&rank=1


C

Could not get review 2 for book 4 searching for "more button": 
https://www.goodreads.com/book/show/25946661-legally-stoned?from_search=true&from_srp=true&qid=xt6dFi55Mw&rank=1


Could not find show more button for Book 4 Review 2: 
https://www.goodreads.com/book/show/25946661-legally-stoned?from_search=true&from_srp=true&qid=xt6dFi55Mw&rank=1


Could not get review 3 for book 4 searching for "more button": 
https://www.goodreads.com/book/show/25946661-legally-stoned?from_search=true&from_srp=true&qid=xt6dFi55Mw&rank=1


Could not find show more button for Book 4 Review 3: 
https://www.goodreads.com/book/show/25946661-legally-stoned?from_search=true&from_srp=true&qid=xt6dFi55Mw&rank=1


Could not get review 4 for book 4 searching for "more button": 
https://www.goodreads.com/book/show/25946661-legally-stoned?from_search=true&from_srp=true&qid=xt6dFi55Mw&rank=1


Could not find show more button for Book 4 Review 4: 
https://www.goodreads.com/book/show/25946661-legally-stoned?from_search

In [17]:
df

,links
0,https://www.goodreads.com/book/show/13771831-b...
1,https://www.goodreads.com/book/show/28111953-j...
2,https://www.goodreads.com/book/show/24612758-m...
3,https://www.goodreads.com/book/show/25946661-l...
4,https://www.goodreads.com/book/show/57566.Secr...
...,...
469,https://www.goodreads.com/book/show/61330903-c...
470,https://www.goodreads.com/book/show/8073047-go...
471,https://www.goodreads.com/book/show/12820471-m...
472,https://www.goodreads.com/book/show/17205.Crys...
